In [17]:
import pandas as pd

from wsdata.consts import ZHUZHAI,BIESHU,SHANGYE,GONGYU,XIEZILOU,BUHANLIGAO,QUANSHI
from wsdata.models import WinsunDatabase
from wsdata.utils import wan, Month

m = Month()

class OneUsage:
    def __init__(self, usage):
        self.usage = usage
        self.ws = WinsunDatabase()
        
    def __gxj(self, window, by, plate=BUHANLIGAO):
        """
        :param window: int
        :param by: 'trend', 'plate'
        """
        return self.ws.gxj(by, 'month', window, usage=self.usage, plate=plate)
    
    @property
    def trend_table(self):
        window = 13
        df = self.__gxj(window, 'trend').df_adjusted
        # df.index = [x[2:] for x in df.index]
        return df
    
    @property
    def trend_argue(self):
        shuoli = self.__gxj(13, 'trend').shuoli(0, 12).full_text
        shuoli = shuoli.replace('增加', '+').replace('下降', '-')
        return shuoli
    
    @property
    def plate_table(self):
        window = 1
        df = self.__gxj(1, 'plate', plate=QUANSHI).df_adjusted
#         df.fillna(0, inplace=True)
        return df
    
    def __top3(self, by):
        """
        :param by: 'sale', 'sold'
        """
        def top(groupby, plate=QUANSHI):
            plate = '仙西' if plate=='仙林' else plate
            df =  self.ws.rank(f'month_{by}', 1, groupby, '面积', usage=self.usage, plate=plate).head(3)
            df.面积 = wan(df.面积)
            return df
        
        # 右表
        plate_top = top('板块')
                
        # 左表
        frames = (top(['板块', 'popularizename'], plate) for plate in plate_top.板块)
        prj_top = pd.concat(frames)
                
        # join
        df = pd.merge(prj_top, plate_top, on='板块')
        df.板块 = df[['板块','面积_y']].apply(lambda x: f'{x.板块}（{x.面积_y}万㎡）', axis=1)
        df.set_index(['板块', 'popularizename'], inplace=True)
        df.drop(['rank_x', 'rank_y', '面积_y'], axis=1, inplace=True)
        
        return df
    
    @property
    def plate_sale_top(self):
        return self.__top3('sale')
    
    @property
    def plate_sold_top(self):
        return self.__top3('sold')
    
    def rank(self, by):
        """
        :param by: 'money', 'space'
        """
        outputs = ['面积','件数','金额','均价'] if by=='space' else ['金额','件数','面积','均价']
        head = 10
        df = self.ws.rank('month_sold', 1, ['popularizename','板块'], outputs, usage=self.usage).head(head)
        df.金额 = wan(df.金额)
        if self.usage==ZHUZHAI:
            df.面积 = wan(df.面积)
            df.金额 = wan(df.金额)
            
        df.均价 = df.均价.astype(int)
        df.set_index('rank', inplace=True)
        return df[['popularizename','板块','面积','件数','金额','均价']]



In [20]:
usage = XIEZILOU
ou = OneUsage(usage)

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


In [21]:
ou.rank('money')

,popularizename,板块,面积,件数,金额,均价
rank,,,,,,
1,万达茂,仙林,15589.15,328,23580.43,15126
2,绿地之窗,城南,5369.24,58,11801.73,21980
3,新地中心,河西,3953.35,15,9495.32,24018
4,证大喜马拉雅中心,城南,4664.64,36,8059.60,17278
5,金奥国际中心,河西,1889.02,11,5349.50,28319
6,南京涵碧楼,河西,1744.43,5,3692.40,21167
7,苏宁慧谷,河西,1222.58,6,2869.34,23470
8,世茂城品,城南,1395.43,7,2560.63,18350
9,涟城,河西,725.07,10,1570.75,21663


In [87]:
ou.trend_table

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


,sale,sold,price
201705,1.34,2.77,21268.0
201706,1.01,10.58,20893.0
201707,NaN,2.43,17883.0
201708,3.00,7.57,17732.0
201709,9.73,0.86,19471.0
201710,5.74,2.01,19809.0
201711,7.60,2.23,20278.0
201712,0.48,5.86,22649.0
201801,12.30,1.91,18166.0
201802,8.29,2.39,18615.0


In [88]:
ou.trend_argue

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


'上市3.02万㎡，环比-66%，同比增长125%。成交3.68万㎡，环比增长20%，同比增长33%。成交均价18850元/㎡，环比增长27%，同比-11%。'

In [89]:
ou.plate_table

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


,sale,sold,price
板块,,,
城中,NaN,NaN,NaN
城东,NaN,NaN,NaN
城南,3.02,1.14,19618.0
河西,NaN,0.97,23910.0
城北,NaN,NaN,NaN
仙西,NaN,NaN,NaN
江宁,NaN,NaN,NaN
浦口,NaN,NaN,NaN
江北新区直管区,NaN,NaN,NaN


In [90]:
df = ou.ws.rank('month_sale', 1, 'popularizename', ['面积','件数'], usage=usage)
df.面积 = wan(df.面积)
df.style.hide_index()

rank,popularizename,面积,件数
1,绿地之窗,3.02,338


In [41]:
res = ou.ws.query('month_sold')\
    .filter(date_range=[m.before(12).date, m.date], usage=ZHUZHAI)\
    .group(['年月','面积段'], '件数')
df = res.cut('acreage', [70,90,120,144,180,250], columns='年月')
df.T

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


acreage        70-  70-90  90-120  120-144  144-180  180-250  250+
   年月                                                             
件数 2017-05-01  197   1954    3371     2250      491      261   132
   2017-06-01  289   2062    2613     1191      368      172    78
   2017-07-01  123   1573    2334     1554      361      170    57
   2017-08-01  133    954    1340      967      217      129    40
   2017-09-01   80   1760    2731     2389      414      250    37
   2017-10-01   18   1391    1184     1066      188      102    15
   2017-11-01   28   1685    1975     1075      293      106    53
   2017-12-01   13   2730    3642     3118      770      199    60
   2018-01-01   98   2387    3119     2043      327      127    44
   2018-02-01   26   1145    1195     1185      105       43    27
   2018-03-01   24   1057    1101      793       91       61    27
   2018-04-01   35   1136    1850     1279      305      257    60
   2018-05-01   23   1133    1746     1065      142       93    71

In [56]:
df = pd.read_clipboard(header=None, index_col=0)
df = pd.DataFrame(df.as_matrix().reshape(13,4))
df.index = pd.date_range(m.before(12).date, m.before(-1).date, freq='M')
df.columns = ['面积','金额','套数','均价']
df.style.hide_columns(['面积','金额']).format({'套数':'{:.0f}', '均价':'{:.0f}'})

d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


,套数,均价
2017-05-31 00:00:00,11110,16958
2017-06-30 00:00:00,7819,17881
2017-07-31 00:00:00,7681,19765
2017-08-31 00:00:00,8404,17365
2017-09-30 00:00:00,6726,19617
2017-10-31 00:00:00,5084,18987
2017-11-30 00:00:00,7191,19891
2017-12-31 00:00:00,4977,18178
2018-01-31 00:00:00,4204,18524
2018-02-28 00:00:00,4435,22126


In [70]:
df = pd.read_clipboard().take([0,1,2,3], axis=1)
df

,面积(㎡),金额(万元),套数,均价(元/㎡)
城北,55319.92,127271.37,760,23006.43
城东,28883.76,78683.19,362,27241.33
城南,40978.61,103015.80,529,25138.92
城中,81989.03,269844.74,1288,32912.30
河西,72153.17,258884.75,870,35879.89
仙西,25024.08,70274.80,240,28082.87
江南六区合计,304348.57,907974.65,4049,29833.38


In [75]:
df['rate'] = df.套数 / df.at['江南六区合计', '套数']
print(df.loc[['城中','河西'], 'rate'].sum())
df.style.background_gradient(subset='rate')

0.5329711039762904


,面积(㎡),金额(万元),套数,均价(元/㎡),rate
城北,55319.9,127271,760,23006.4,0.187701
城东,28883.8,78683.2,362,27241.3,0.0894048
城南,40978.6,103016,529,25138.9,0.13065
城中,81989,269845,1288,32912.3,0.318103
河西,72153.2,258885,870,35879.9,0.214868
仙西,25024.1,70274.8,240,28082.9,0.0592739
江南六区合计,304349,907975,4049,29833.4,1
